Datenaufbereitung V2. In Version 1 habe ich gemerkt, dass die Daten nicht einheitlich sind bezüglich der Alter und Beschreibung, warum ein Titel eine Altersbeschränkung von USK oder Steam-Germany bekommen hat. Nun sollte dies einheitlich sein. Gleichzeitg wurden die Spiele nochmals durch das gleiche Programm gefiltert und die Altersfreigabe sowohl für Steam-Germany als auch USK hinzugefügt.

In [1]:
import pandas as pd
import datetime 

In [ ]:
data = pd.read_csv("../data/second_required_info.csv")

In [3]:
data[data["id"]==730]

,id,alter_usk,eigenschaften_usk,alter_steam,eigenschaften,genre,categories,released,original_price,is_free,recommendations
26,730,16.0,Gewalt; In-Game Käufe (+zufällige Objekte); Chats,16.0,Drastische Gewalt,Action; Kostenlos spielbar,Mehrspieler; Plattformübergreifender Mehrspiel...,21. Aug. 2012,NaN,True,4723170.0


In [4]:
data["required_age"] = data["alter_steam"].combine_first(data["alter_usk"]) # only one rating required
len(data)

123699

In [5]:
data = data[data["required_age"].notna()]
len(data)

114274

After I finished the age, i will now go on to the description why a game got its age rating and will combine both  

In [247]:
data[data["id"] == 582660]

,id,alter_usk,eigenschaften_usk,alter_steam,eigenschaften,genre,categories,released,original_price,is_free,recommendations,required_age
8647,582660,16.0,NaN,16.0,Gewalt; Alkohol / Tabak; Sexuelle Andeutungen,Action; Abenteuer; MMO; Rollenspiel; Strategie,Mehrspieler; MMO; PvP; Online-PvP; Koop; Onlin...,24. Mai 2017,999.0,False,60057.0,16.0


In [248]:
data["eigenschaften"] = data["eigenschaften"].fillna("NA")
data["eigenschaften_usk"] = data["eigenschaften_usk"].fillna("NA")
data["feature"] = data["eigenschaften"] +";"+ data["eigenschaften"] 

In [249]:
data["feature"] = data["feature"].str.split(";").apply(lambda x: list(dict.fromkeys(x)))

In [250]:
data["feature"] = data["feature"].apply(lambda x: ";".join(x))

In [251]:
data["feature"] = data["feature"].apply(
    lambda x: x.replace("NA", "").rstrip(";").strip() if len(x) > 2 else x
) # take away the extra na in the combined data if usk or steam-germany had an na and the other not

In [252]:
data["released"].value_counts()

released
Bald verfügbar               12174
Bevorstehende Ankündigung     7928
2025                          1858
2026                          1815
4. Quartal 2025                980
                             ...  
26. Jan. 2026                    1
2029                             1
17. Nov. 2008                    1
1. Jun. 2006                     1
1. Mai 2006                      1
Name: count, Length: 4992, dtype: int64

In [253]:
data[data["id"] == 582660] # Path of exile

,id,alter_usk,eigenschaften_usk,alter_steam,eigenschaften,genre,categories,released,original_price,is_free,recommendations,required_age,feature
8647,582660,16.0,NA,16.0,Gewalt; Alkohol / Tabak; Sexuelle Andeutungen,Action; Abenteuer; MMO; Rollenspiel; Strategie,Mehrspieler; MMO; PvP; Online-PvP; Koop; Onlin...,24. Mai 2017,999.0,False,60057.0,16.0,Gewalt; Alkohol / Tabak; Sexuelle Andeutungen


In [254]:
# Q, Zahl, Leerzeichen(optional), 4 stellige dezimal, Monate, und der andere Quatsch
pattern = r"(Quartal|Januar|Februar|März|April|Juni|July|August|September|Oktober|November|Dezember|Bald verfügbar|Bevorstehende Ankündigung)"

data = data[~data["released"].str.contains(pattern, case=False, na=False)]

mask = data["released"].astype(str).str.fullmatch(r"\d{4}")

data = data[~mask]

data = data.drop(data[data["released"].isna()].index)



C:\Users\Flo\AppData\Local\Temp\ipykernel_7872\4132130075.py:4: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  data = data[~data["released"].str.contains(pattern, case=False, na=False)]


In [255]:
data["released"].value_counts()

released
19. Jun. 2025    146
31. Okt. 2024    135
31. Jul. 2025    125
25. Okt. 2024    124
1. Aug. 2024     118
                ... 
13. Jan. 2026      1
11. Feb. 2026      1
21. Dez. 2025      1
15. Nov. 2025      1
17. Nov. 2008      1
Name: count, Length: 4943, dtype: int64

In [256]:
# 1. Deutsche Monatskürzel durch englische ersetzen

data["released"] = data["released"].str.replace("Mrz", "Mar", regex=False)
data["released"] = data["released"].str.replace("Mai", "May", regex=False)
data["released"] = data["released"].str.replace("Okt", "Oct", regex=False)
data["released"] = data["released"].str.replace("Dez", "Dec", regex=False)
data["released"] = data["released"].str.replace(".", "", regex=False)


data[data["id"] == 582660]

,id,alter_usk,eigenschaften_usk,alter_steam,eigenschaften,genre,categories,released,original_price,is_free,recommendations,required_age,feature
8647,582660,16.0,NA,16.0,Gewalt; Alkohol / Tabak; Sexuelle Andeutungen,Action; Abenteuer; MMO; Rollenspiel; Strategie,Mehrspieler; MMO; PvP; Online-PvP; Koop; Onlin...,24 May 2017,999.0,False,60057.0,16.0,Gewalt; Alkohol / Tabak; Sexuelle Andeutungen


In [257]:
# Datetime conversion
data["released"] = pd.to_datetime(
    data["released"],
    format="mixed",    
    errors="coerce"
)

In [258]:
data = data[data["released"] < datetime.datetime.now()] 

In [259]:
data.loc[data["is_free"] == True, "original_price"] = 0


In [260]:
data[data["id"] == 582660]

,id,alter_usk,eigenschaften_usk,alter_steam,eigenschaften,genre,categories,released,original_price,is_free,recommendations,required_age,feature
8647,582660,16.0,NA,16.0,Gewalt; Alkohol / Tabak; Sexuelle Andeutungen,Action; Abenteuer; MMO; Rollenspiel; Strategie,Mehrspieler; MMO; PvP; Online-PvP; Koop; Onlin...,2017-05-24,999.0,False,60057.0,16.0,Gewalt; Alkohol / Tabak; Sexuelle Andeutungen


In [ ]:
# import matching names and get one dataframe


names = pd.read_csv("../data/spiele_ids_steam.csv")
names[names["appid"] == 582660]

,appid,name
33082,582660,Black Desert


In [262]:
joined_data = pd.merge(data, names , left_on="id", right_on="appid")

In [263]:
joined_data[joined_data["id"] == 582660]

,id,alter_usk,eigenschaften_usk,alter_steam,eigenschaften,genre,categories,released,original_price,is_free,recommendations,required_age,feature,appid,name
8287,582660,16.0,NA,16.0,Gewalt; Alkohol / Tabak; Sexuelle Andeutungen,Action; Abenteuer; MMO; Rollenspiel; Strategie,Mehrspieler; MMO; PvP; Online-PvP; Koop; Onlin...,2017-05-24,999.0,False,60057.0,16.0,Gewalt; Alkohol / Tabak; Sexuelle Andeutungen,582660,Black Desert


In [264]:
wanted_information = joined_data[['id',"name","released","original_price","genre","categories","required_age","feature", "recommendations"]]

In [265]:
wanted_information.head()

,id,name,released,original_price,genre,categories,required_age,feature,recommendations
0,10,Counter-Strike,2000-11-01,819.0,Action,Mehrspieler; PvP; Online-PvP; PvP-Spiele mit g...,16.0,Drastische Gewalt,162690.0
1,20,Team Fortress Classic,1999-04-01,499.0,Action,Mehrspieler; PvP; Online-PvP; PvP-Spiele mit g...,12.0,Gewalt,6680.0
2,30,Day of Defeat,2003-05-01,499.0,Action,Mehrspieler; Kamerakomfort; Farbalternativen; ...,16.0,Drastische Gewalt,4328.0
3,40,Deathmatch Classic,2001-06-01,499.0,Action,Mehrspieler; PvP; Online-PvP; PvP-Spiele mit g...,6.0,Fantasy-Gewalt,2349.0
4,50,Half-Life: Opposing Force,1999-11-01,499.0,Action,Einzelspieler; Mehrspieler; Benutzerdefinierte...,16.0,Drastische Gewalt,23028.0


In [266]:
wanted_information= wanted_information.rename(columns={
    "recommendations": "total_steam_reviews_with_game",
    "original_price": "price_in_cents_no_discount"
})

In [ ]:
wanted_information.to_csv("../data/wanted_information.csv")

In [268]:
all_genres = []

# NaN in der genre-Spalte entfernen
genres = wanted_information["genre"].fillna("").str.split(";")

# über jeden Zeilenwert iterieren
for genre_list in genres:
    for genre in genre_list:
        genres = genre.strip()
        if genres and genres not in all_genres:
            all_genres.append(genres)

len(all_genres)


33

In [ ]:
pd.Series(all_genres, name="genres").to_csv("../data/alle_genres_bei_steam.csv")

In [270]:
all_categories = []

categories = wanted_information["categories"].fillna("").str.split(";")

for category_list in categories:
    for category in category_list:
        cat = category.strip()
        if cat and cat not in all_categories:  
            all_categories.append(cat)

len(all_categories)


58

In [ ]:

pd.Series(all_categories, name="categories").to_csv("../data/alle_categories_bei_steam.csv")

In [272]:
wanted_information[wanted_information["id"]== 582660]

,id,name,released,price_in_cents_no_discount,genre,categories,required_age,feature,total_steam_reviews_with_game
8287,582660,Black Desert,2017-05-24,999.0,Action; Abenteuer; MMO; Rollenspiel; Strategie,Mehrspieler; MMO; PvP; Online-PvP; Koop; Onlin...,16.0,Gewalt; Alkohol / Tabak; Sexuelle Andeutungen,60057.0
